<H1>Week 3: Capstone Project</H1>
<H4>Alejandro Plaza Larrea</H4>


<p> First step is to import the table from Wikipedia. 
    I will use BeautifulSoup to extract the HTML text
    from the Wikipedia page and then convert it to a df.</p>

In [20]:
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"lxml")
#print(soup.prettify())

In [21]:
My_table = soup.find("table",{"class":"wikitable sortable"})
#My_table

In [22]:
import pandas as pd
dfs = pd.read_html(str(My_table))
#dfs

In [23]:
df = dfs[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


<p> I will remove the "not assigned" Borough... </p>

In [24]:
df = df[df.Borough != "Not assigned"]
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


<p> Since there are not "not assigned" Neighborhood, I didn't replace them with the Borough </p>

In [25]:
df[df.Neighbourhood == "Not assigned"]

,Postcode,Borough,Neighbourhood


In [26]:
index = df.Postcode
index.value_counts()

M9V    8
M8Y    8
M5V    7
M8Z    5
M9B    5
      ..
M9L    1
M3B    1
M5N    1
M9W    1
M5C    1
Name: Postcode, Length: 103, dtype: int64

In [27]:
df["Neighbourhood"] = df.groupby("Postcode")["Neighbourhood"].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
...,...,...,...
254,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
261,M4Y,Downtown Toronto,Church and Wellesley
264,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
265,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [28]:
df.shape

(103, 3)

In [29]:
geo_df=pd.read_csv("https://cocl.us/Geospatial_data")
geo_df.columns = ["Postcode","Latitude","Longitude"]
geo_df

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [30]:
df = pd.merge(df,geo_df, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [32]:
conda install -c conda-forge folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    conda-4.8.2                |           py37_0         3.0 MB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.0-py_

In [33]:
import folium

In [39]:
latitude = 43
longitude = -79
map = folium.Map(location=(latitude,longitude), zoom_start=10)

for lat, longi, borough, neigh in zip(df["Latitude"], 
                                      df["Longitude"],
                                      df["Borough"], 
                                      df["Neighbourhood"]):
    label = '{},{}'.format(neigh,borough)
    label = folium.Popup(label,parse_HTML=True)
    folium.CircleMarker(
        [lat,longi],
        radius = 4,
        popup = label,
        color = "blue",
        fill = True,
        fill_color = "grey",
        fill_opacity = 0.7,
        parse_html = False
    ).add_to(map)
map